In [ ]:
# import pandas as pd
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# # Load dataset
# df_news = pd.read_csv("/home/rupam/DataAlpha/data/raw/News_Headline/abcnews-date-text.csv")

# # Rename for consistency if needed
# df_news.columns = df_news.columns.str.strip()

# # Check actual column names and types
# print(df_news.dtypes)

# # Convert 'publish_date' from YYYYMMDD integer/string to datetime
# df_news["publish_date"] = pd.to_datetime(df_news["publish_date"], format="%Y%m%d", errors="coerce")

# # Filter for 2020 only
# df_news_2020 = df_news[df_news["publish_date"].dt.year == 2020].copy()

# # Drop rows with missing headlines
# df_news_2020 = df_news_2020[df_news_2020["headline_text"].notna()]

# # Optional: Save the cleaned file
# df_news_2020.to_csv("/home/rupam/DataAlpha/data/raw/News_Headline/processed_news_2020.csv", index=False)

# # Check min/max date again
# min_date = df_news_2020["publish_date"].min()
# max_date = df_news_2020["publish_date"].max()

# print(f"Earliest article date: {min_date.date()}")
# print(f"Latest article date:   {max_date.date()}")
# df_news_2020.shape


publish_date      int64
headline_text    object
dtype: object
Earliest article date: 2020-01-01
Latest article date:   2020-12-31


(40240, 2)

In [ ]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyzer = SentimentIntensityAnalyzer()

# # Use .loc to avoid SettingWithCopyWarning
# df_news_2020.loc[:, "sentiment"] = df_news_2020["headline_text"].apply(
#     lambda x: analyzer.polarity_scores(str(x))["compound"]
# )
# output_path = "/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv"
# df_news_2020.to_csv(output_path, index=False)

# # Confirm
# print(f"✅ Sentiment-enriched CSV saved to:\n{output_path}")
# print(df_news_2020[["publish_date", "headline_text", "sentiment"]].head())

# df_news_2020.shape

✅ Sentiment-enriched CSV saved to:
/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv
Empty DataFrame
Columns: [publish_date, headline_text, sentiment]
Index: []


(0, 4)

In [ ]:
# import pandas as pd
# import re

# # Load your filtered 2020 news data with sentiment
# df_news_2020 = pd.read_csv("/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv")

# # Load S&P 500 tickers from Wikipedia
# sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
# sp500_tickers = set(sp500["Symbol"].tolist())

# # Function to extract S&P 500 tickers from headline text
# def extract_tickers(text):
#     words = re.findall(r"\b[A-Z]{1,5}\b", str(text))
#     return [w for w in words if w in sp500_tickers]

# # Apply extraction
# df_news_2020["tickers"] = df_news_2020["headline_text"].apply(extract_tickers)

# # Keep only rows with at least one ticker mentioned
# df_news_2020 = df_news_2020[df_news_2020["tickers"].map(len) > 0]

# # Reset index (optional)
# df_news_2020 = df_news_2020.reset_index(drop=True)

# # Save cleaned file
# df_news_2020.to_csv("news_sentiment_2020_filtered.csv", index=False)

# # Preview
# print(df_news_2020.head())
# print(f"Number of filtered articles: {len(df_news_2020)}")


Empty DataFrame
Columns: [publish_date, headline_text, sentiment, tickers]
Index: []
Number of filtered articles: 0


In [23]:
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# -----------------------------
# 1. Load the ABC News dataset
# -----------------------------
df_news = pd.read_csv("/home/rupam/DataAlpha/data/raw/News_Headline/abcnews-date-text.csv")

# Fix column names
df_news.columns = df_news.columns.str.strip()

# Convert publish_date to datetime (format is YYYYMMDD)
df_news["publish_date"] = pd.to_datetime(df_news["publish_date"], format="%Y%m%d", errors="coerce")

# Filter for year 2020
df_news_2020 = df_news[df_news["publish_date"].dt.year == 2020].copy()

# Drop empty headlines
df_news_2020 = df_news_2020[df_news_2020["headline_text"].notna()]

# -----------------------------
# 2. Load S&P 500 Company Names and Tickers
# -----------------------------
wiki_table = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

# Create a mapping from lowercase company name to ticker
company_to_ticker = {
    row["Security"].lower(): row["Symbol"]
    for _, row in wiki_table.iterrows()
}

company_names = list(company_to_ticker.keys())

# -----------------------------
# 3. Match company name in headline
# -----------------------------
def extract_tickers_from_company_names(headline):
    headline_lower = headline.lower()
    matched = []
    for company in company_names:
        if company in headline_lower:
            matched.append(company_to_ticker[company])
    return list(set(matched))  # remove duplicates

df_news_2020["tickers"] = df_news_2020["headline_text"].apply(extract_tickers_from_company_names)

# Keep only rows with at least one ticker match
df_news_2020 = df_news_2020[df_news_2020["tickers"].map(len) > 0].copy()

# -----------------------------
# 4. Apply VADER Sentiment
# -----------------------------
analyzer = SentimentIntensityAnalyzer()
df_news_2020["sentiment"] = df_news_2020["headline_text"].apply(
    lambda x: analyzer.polarity_scores(str(x))["compound"]
)

# -----------------------------
# 5. Expand to one row per ticker
# -----------------------------
expanded_rows = []
for _, row in df_news_2020.iterrows():
    for ticker in row["tickers"]:
        expanded_rows.append({
            "date": row["publish_date"].date(),
            "ticker": ticker,
            "sentiment": row["sentiment"],
            "headline": row["headline_text"]
        })

df_expanded = pd.DataFrame(expanded_rows)

# -----------------------------
# 6. Aggregate sentiment by date & ticker
# -----------------------------
df_sentiment_daily = df_expanded.groupby(["date", "ticker"]).agg({
    "sentiment": "mean"
}).reset_index()

# -----------------------------
# 7. Save final CSV
# -----------------------------
output_path = "/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv"
df_sentiment_daily.to_csv(output_path, index=False)

print(f"\n✅ Saved final sentiment file: {output_path}")
print(df_sentiment_daily.head())



✅ Saved final sentiment file: /home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv
         date ticker  sentiment
0  2020-01-03   AMZN    -0.1779
1  2020-01-08     BA    -0.4010
2  2020-01-12   AMZN     0.1779
3  2020-01-12     BA     0.0000
4  2020-01-12    MMM     0.0000


In [25]:
print(f"Total unique tickers matched in news: {df_sentiment_daily['ticker'].nunique()}")
print("Some of the tickers found:", df_sentiment_daily['ticker'].unique()[:28])


Total unique tickers matched in news: 28
Some of the tickers found: ['AMZN' 'BA' 'MMM' 'NFLX' 'CMI' 'MSFT' 'CSCO' 'ABNB' 'WFC' 'UBER' 'MAS'
 'EFX' 'CCL' 'INTC' 'GM' 'APTV' 'COST' 'HOLX' 'DD' 'PFE' 'MRNA' 'STE'
 'AMCR' 'GILD' 'WM' 'LEN' 'LDOS' 'WMT']
